# 📝 Ecommerce AI Agent (LangGraph)

---

## Overview

- This notebook/script builds an advanced **ecommerce customer service AI agent**.
- The agent is **modular**, able to answer FAQs, order status, refund, returns, and review queries.
- **Semantic FAQ retrieval** is powered by embeddings + ChromaDB.
- **Structured data** (orders, payments, reviews) is retrieved from your SQL database.
- **Intent classification** is done using an LLM (OpenAI or open-source via HuggingFace).
- All user **Q&A pairs are logged** for learning, retraining, and analytics.
- **Workflow and tool orchestration** are handled by LangGraph for multi-step reasoning.

---

## Code Section Guide

1. **Imports**  
   _All required Python libraries are loaded here._

2. **FAQ Vector Store**  
   _Loads FAQ data, builds a semantic search index._

3. **Database Connection**  
   _Connects to your ecommerce database._

4. **LLM Setup**  
   _Configures LLM for intent classification (OpenAI or HuggingFaceHub)._

5. **Tool Definitions**  
   _Functions for each business operation (FAQ, order, refund, return, review)._

6. **Memory**  
   _Initializes a conversation buffer._

7. **LangGraph Nodes**  
   _Defines workflow steps for multi-step reasoning._

8. **Build Workflow**  
   _Wires agent steps together using LangGraph._

9. **Chat Loop**  
   _Interactive chat session (replace with API in production)._

---

## Customization/Extension Tips

- **Add more tools:**  
  Define new functions and update both the tools list and tool_node routing.

- **Improve returns/refund logic:**  
  Expand those functions to match your real processes and schema.

- **Change LLM or database:**  
  Update config and environment variables as needed.

- **Deploy as API:**  
  Replace the CLI chat loop with a FastAPI or Flask route.

---

## Security/Production Tips

- **Do NOT hardcode credentials.** Use environment variables.
- **Check and filter logs** for PII or sensitive info.
- **Sanitize all user input** that hits the database.

---

## Further Reading

- [LangChain Documentation](https://python.langchain.com/)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [ChromaDB Documentation](https://docs.trychroma.com/)
- [HuggingFace Hub](https://huggingface.co/)

---

## Questions?

See the inline comments and docstrings, or ping the code owner for guidance!

---

## Ecommerce AI Agent (LangGraph) — Jupyter Notebook Guide

## Overview

- Multi-tool, multi-intent AI agent for ecommerce support (LangChain + LangGraph).
- Supports FAQ, order status, refund, return, reviews (semantic + DB).
- Uses LLM for intent classification.
- Every Q&A logged for future learning.
- Easily extendable (more tools, APIs, memory).


In [1]:
# Install Dependencies
# !pip install -q langchain langgraph langchain_community langchain_chroma chromadb sqlalchemy datasets openai langchain_huggingface huggingface_hub

# =======================
# 1. Setup: Imports
# =======================
import os
import datetime
import shutil

from datasets import load_dataset
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEndpoint
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langgraph.checkpoint.memory import InMemorySaver
from openai import OpenAI

from sqlalchemy import create_engine, MetaData

from langgraph.graph import StateGraph

import pandas as pd
import sqlite3
from sqlalchemy import Table, Column, Integer, String, Float, MetaData
from dotenv import load_dotenv

In [2]:
# =======================
# 2. FAQ Vector Store Setup
# =======================
# Loads FAQ data from Hugging Face and builds a semantic search vector DB (Chroma).

# ---
# Optional: Delete vector DB folder if you want to recreate it from scratch
VECTOR_DB_DIR = "./faq_vectorstore"
# if os.path.exists(VECTOR_DB_DIR):
#     shutil.rmtree(VECTOR_DB_DIR)
# ---
faq_dataset = load_dataset("Andyrasika/Ecommerce_FAQ")['train']
faqs = [f"{row['question']} {row['answer']}" for row in faq_dataset]
ids = [str(i) for i in range(len(faqs))]
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma(
    collection_name="faq",
    embedding_function=embeddings,
    persist_directory=VECTOR_DB_DIR
)

In [3]:
# Check if DB is empty
if not vector_db.get()['ids']:
    print("Adding FAQs to vector store...")
    vector_db.add_texts(faqs, ids=ids)
    print("Vector store persisted.")
else:
    print("Vector store already exists. Skipping addition.")

Vector store already exists. Skipping addition.


In [4]:
# =======================
# 3. Database Connection (from CSVs if no DB exists)
# =======================
# Loads CSVs into a fresh SQLite DB if not present, or connects directly if DB file already exists.
DATABASE_FILE = "olist.db" 
DATABASE_URL = f"sqlite:///{DATABASE_FILE}"
FOLDER = "./data"

# Specify your CSV files and corresponding table names here
csv_table_map = {
    'olist_customers_dataset' : 'olist_customers_dataset.csv',
    'olist_geolocation_dataset' : 'olist_geolocation_dataset.csv',
    'olist_orders_dataset' : 'olist_orders_dataset.csv',
    'olist_order_items_dataset' : 'olist_order_items_dataset.csv',
    'olist_order_payments_dataset' : 'olist_order_payments_dataset.csv',
    'olist_order_reviews_dataset' : 'olist_order_reviews_dataset.csv',
    'olist_products_dataset' : 'olist_products_dataset.csv',
    'olist_sellers_dataset' : 'olist_sellers_dataset.csv',
    'product_category_name_translation' : 'product_category_name_translation.csv' 
}

def create_db_from_csvs(csv_table_map, db_file, folder):
    """If db_file doesn't exist, create it and import all CSVs as tables."""
    if os.path.exists(db_file):
        print(f"Database:{db_file} already exists")
        return None
    conn = sqlite3.connect(db_file)
    for table, csv in csv_table_map.items():
        if not os.path.exists(f"{folder}/{csv}"):
            print(f"Warning: {folder}/{csv} not found. Skipping table {table}.")
            continue
        else:
            print(f"Writing: {folder}/{csv} into {table}.")
        df = pd.read_csv(f"{folder}/{csv}")
        df.to_sql(table, conn, index=False, if_exists='replace')
    conn.close()

# Only create DB from CSVs if DB file does not exist
create_db_from_csvs(csv_table_map, DATABASE_FILE, FOLDER)

# Now connect via SQLAlchemy as usual
engine = create_engine(DATABASE_URL)
metadata = MetaData()
metadata.reflect(bind=engine)

# You can now access your tables as before!
olist_customers = metadata.tables.get('olist_customers_dataset')
olist_geolocation = metadata.tables.get('olist_geolocation_dataset')
olist_orders = metadata.tables.get('olist_orders_dataset')
olist_order_items = metadata.tables.get('olist_order_items_dataset')
olist_order_payments = metadata.tables.get('olist_order_payments_dataset')
olist_order_reviews = metadata.tables.get('olist_order_reviews_dataset')
olist_products = metadata.tables.get('olist_products_dataset')
olist_sellers =	metadata.tables.get('olist_sellers_dataset')
product_category_name_translation = metadata.tables.get('product_category_name_translation')

Database:olist.db already exists


In [5]:
# =======================
# 4. LLM Setup (with .env support)
# =======================
# Supports OpenAI and HuggingFace models. Reads API keys from .env via python-dotenv.
load_dotenv()  # Automatically loads variables from a .env file at project root

def llm(prompt):
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages= prompt)    
    intent = response.choices[0].message.content.strip().lower()
    return intent

# .env example (do NOT check your keys into version control):
# OPENAI_API_KEY=sk-...your-openai-key...
# HUGGINGFACEHUB_API_TOKEN=hf_...your-hf-token...

In [6]:
# =======================
# 5. Tool Definitions
# =======================
# Each function is a "tool" for one business task.
# Tools use regex to extract elements from user queries.
def search_faq(query):
    """Semantic search in FAQ vector DB."""
    docs = vector_db.similarity_search(query, k=2)
    return "\n".join([doc.page_content for doc in docs]) if docs else "No relevant FAQ found."

faq_tool = Tool(
    name="faq_search",
    func=search_faq,
    description="Semantic search in FAQ for general questions."
)   

def get_order_status(query):
    """Retrieve order status from DB using order_id."""
    import re
    match = re.search(r'(\b[0-9a-f]{32,}\b)', query)
    order_id = match.group(1) if match else None
    if order_id and olist_orders is not None:
        with engine.connect() as conn:
            res = conn.execute(
                olist_orders.select().where(olist_orders.c.order_id == order_id)
            ).fetchone()
            if res:
                return f"Order {order_id} status: {res.order_status}, purchased: {res.order_purchase_timestamp}, estimated delivery: {res.order_estimated_delivery_date}"
            else:
                return f"No order found for ID: {order_id}."
    else:
        return "Please provide a valid order ID."

order_status_tool = Tool(
    name="order_status_lookup",
    func=get_order_status,
    description="Look up order status by order_id."
)

def get_refund_status(query):
    """Check refund/payment info for an order."""
    import re
    match = re.search(r'(\b[0-9a-f]{32,}\b)', query)
    order_id = match.group(1) if match else None
    payments_table = metadata.tables.get('olist_order_payments_dataset')
    if not order_id or payments_table is None:
        return "Please provide a valid order ID."
    with engine.connect() as conn:
        res = conn.execute(
            payments_table.select().where(payments_table.c.order_id == order_id)
        ).fetchall()
        if not res:
            return f"No payment info for order {order_id}."
        total_paid = sum(r.payment_value for r in res)
        if total_paid == 0:
            return f"Order {order_id} was fully refunded."
        else:
            return f"Order {order_id} was paid {total_paid}, refund status unknown."

refund_status_tool = Tool(
    name="refund_status_lookup",
    func=get_refund_status,
    description="Check if an order has been refunded by order_id."
)

def get_review(query):
    """Retrieve review score/message for an order."""
    import re
    match = re.search(r'(\b[0-9a-f]{32,}\b)', query)
    order_id = match.group(1) if match else None
    reviews_table = metadata.tables.get('olist_order_reviews_dataset')
    if not order_id or reviews_table is None:
        return "Please provide a valid order ID."
    with engine.connect() as conn:
        res = conn.execute(
            reviews_table.select().where(reviews_table.c.order_id == order_id)
        ).fetchone()
        if not res:
            return f"No review found for order {order_id}."
        return f"Review for order {order_id}: Score {res.review_score} - {res.review_comment_message or 'No comment.'}"

review_tool = Tool(
    name="review_lookup",
    func=get_review,
    description="Retrieve review and score for an order by order_id."
)

tools = [faq_tool, order_status_tool, refund_status_tool, review_tool]

In [7]:
# =======================
# 6. Memory
# =======================
checkpointer = InMemorySaver()  # Keeps conversation state in memory (can be swapped for DB, Redis, etc.)

In [8]:
# =======================
# 7. LangGraph Nodes (Workflow)
# =======================
def perception_node(state):
    """LLM-based intent classification."""
    query = state["input"]
    prompt = [
        {"role": "system", "content": "You are a helpful intent classifier."},
        {"role": "user", "content": (
            "Classify the user's query into one of these intents: "
            "faq, order_status, refund_status, review. "
            "Respond with only the intent word, nothing else. "
            f"Here is the query: {query}"
        )}
    ]
    intent = llm(prompt).strip().lower()
    allowed = {"faq", "order_status", "refund_status", "review"}
    if intent not in allowed:
        if "order" in query:
            intent = "order_status"
        else:
            intent = "faq"
    state["classification"] = intent
    return state

In [9]:
def tool_node(state):
    """Dispatch to the correct business logic tool based on classified intent."""
    classification = state["classification"]
    query = state["input"]
    if classification == "order_status":
        state["tool_output"] = get_order_status(query)
    elif classification == "faq":
        state["tool_output"] = search_faq(query)
    elif classification == "refund_status":
        state["tool_output"] = get_refund_status(query)
    elif classification == "return_status":
        state["tool_output"] = get_return_status(query)
    elif classification == "review":
        state["tool_output"] = get_review(query)
    else:
        state["tool_output"] = "I'm sorry, I could not classify your request."
    return state

def answer_node(state):
    """Wrap up the result for agent response."""
    answer = state["tool_output"]
    state["output"] = answer
    return state

def log_interaction(user_query, agent_answer):
    """Log each Q&A for learning, retraining, or analytics."""
    with open("agent_interactions.log", "a", encoding="utf-8") as f:
        f.write(f"{datetime.datetime.now().isoformat()} | Q: {user_query} | A: {agent_answer}\n")

def learning_node(state):
    """Learning step: log output for future improvement."""
    log_interaction(state["input"], state["output"])
    return state

In [10]:
# =======================
# 8. Build LangGraph Workflow (with InMemorySaver checkpointing)
# =======================
from typing import TypedDict

# Define the state schema required for StateGraph as a TypedDict (not a plain dict!)
class AgentState(TypedDict):
    input: str
    classification: str
    tool_output: str
    output: str

workflow = StateGraph(AgentState)
workflow.add_node("perception", perception_node)
workflow.add_node("tool_use", tool_node)
workflow.add_node("answer", answer_node)
workflow.add_node("learning", learning_node)
# The magic fix: add edge from __start__ to your entry node!
workflow.add_edge("__start__", "perception")
workflow.add_edge("perception", "tool_use")
workflow.add_edge("tool_use", "answer")
workflow.add_edge("answer", "learning")

# For latest LangGraph, entry/finish nodes are set in compile as positional args!
graph = workflow.compile()

In [ ]:
# =======================
# 9. Chat Loop
# =======================
def ask_agent(user_query):
    state = {"input": user_query}
    result = graph.invoke(state)
    return result["output"]

print("Ecommerce AI Agent (type 'quit' to exit).")
while True:
    q = input("You: ")
    if q.lower() in {"quit", "exit"}:
        break
    print("AI:", ask_agent(q))

Ecommerce AI Agent (type 'quit' to exit).


You:  What is the review for 00010242fe8c5a6d1ba2dd792cb16214?


AI: Review for order 00010242fe8c5a6d1ba2dd792cb16214: Score 5 - Perfeito, produto entregue antes do combinado.
